## Despliegue de la base de datos

Ejecutar en terminal con docker instalado

In [ ]:
docker pull pgvector/pgvector:0.8.0-pg17

docker run -e POSTGRES_USER=myuser            -e POSTGRES_PASSWORD=mypassword            -e POSTGRES_DB=mydatabase            --name my_postgres            -p 5432:5432            -d pgvector/pgvector:0.8.0-pg17

## Inicialización de la base de datos

En primer lugar, inicializamos la DB postgres y añadimos la extensión para vectores

In [ ]:
%pip install psycopg2

In [38]:
import psycopg2

# Connect to your database
conn = psycopg2.connect(
    dbname="mydatabase",
    user="myuser",
    password="mypassword",
    host="localhost"
)

In [39]:



# Create the 'vector' extension if it doesn't exist
with conn.cursor() as cur:
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    conn.commit()

# Verify the extension is installed
with conn.cursor() as cur:
    cur.execute("SELECT * FROM pg_extension;")
    extensions = cur.fetchall()
    print("Installed extensions:", extensions)

Installed extensions: [(13569, 'plpgsql', 10, 11, False, '1.0', None, None), (16385, 'vector', 10, 2200, True, '0.8.0', None, None)]


Conectamos con la DB y creamos una tabla "chunks" donde vamos a almacenar los fragmentos del texto
Como vamos a usar los vectores de openai cuya dimensión es 3072, lo configuramos.

In [ ]:

# Create a table with a vector column
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE chunks (
            id serial PRIMARY KEY,
            texto_original TEXT,
            empresa TEXT,
            numero_pagina INTEGER,
            embedding vector(3072)
        );
    """)
    conn.commit()

## Extracción e Ingesta

### Llamamos extracción al proceso de leer de los pdfs y transformar la información a lenguaje natural.
### Llamamos ingesta a vectorizar los documentos según una estrategia de troceado y a insertarlos tras su vectorización en la base de datos.

In [ ]:
%pip install -qU pypdf 
%pip install langchain-community
%pip install tqdm 

Para este ejemplo vamos a usar PyPDF para la extracción y langchain para simplificar las llamadas. Al hacerlo asíncrono lo hacemos mucho mas rápido.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from tqdm.asyncio import tqdm

loader = PyPDFLoader("jga25-cuentas-anuales-consolidadas-2024.pdf")
pages = []
async for page in tqdm(loader.alazy_load(), desc="Loading pages"):
    pages.append(page)

In [ ]:
print(page)

Vamos a convertir el formato en un objeto similar a los campos de nuestra BD, tambien limpiamos algunas páginas que no tienen texto

In [ ]:
class Pagina:
    def __init__(self, texto_fuente, pagina):
        self.texto_fuente = texto_fuente
        self.empresa = "Iberdrola"
        self.pagina = pagina
        self.vector = []
    def __str__(self):
        return f"Pagina {self.pagina}: {self.texto_fuente}"

In [ ]:


# Convert the array of pages into an array of Pagina objects, excluding empty page_content
paginas = [Pagina(page.page_content, page.metadata['page']) for page in pages if page.page_content]

# Print the third Pagina object
print(paginas[90])

## Ingesta

Instalamos el paquete langchain openai para hacer la vectorización de los pdfs

In [ ]:
%pip install -qU langchain-openai 

Desplegamos un modelo en azure openai de embeddings y lo configuramos

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint="",
    api_key="",
    api_version="2024-12-01-preview"
)

Vectorizamos la pagina 15 y lo imprimimos para comprobar que funciona


In [ ]:
vector = embeddings.embed_query(paginas[15].texto_fuente)
print(paginas[15], vector)

Vectorizamos todas las paginas y guardamos el vector resultante en el objeto que ya habiamos creado, usamos tqdm para mostrar el progreso
Si tuvieramos una gran cantidad de documentos, sería muy recomendable hacer el proceso asíncrono y paralelizarlo.

In [ ]:
from tqdm.asyncio import tqdm

for pagina in tqdm(paginas, desc="Vectorizando paginas"):
    pagina.vector = embeddings.embed_query(pagina.texto_fuente)


Una vez vectorizados, hacemos la inserción en la base de datos.

In [ ]:
with conn.cursor() as cur:
    for pagina in paginas:
        cur.execute("""
            INSERT INTO chunks (texto_original, empresa, numero_pagina, embedding)
            VALUES (%s, %s, %s, %s)
        """, (pagina.texto_fuente, pagina.empresa, pagina.pagina, pagina.vector))
    conn.commit()


Comprobamos que el dato quedó bien

In [ ]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM chunks;")
    rows = cur.fetchone()
    print(rows)

## Retrieval
Conocemos retrieval como el proceso de busqueda de la información relevante, esta información es la que se insertará en un prompt para que el LLM pueda dar respuestas contextualizadas

In [ ]:
#Pedimos al usuario una pregunta y la vectorizamos
consulta = input("Introduce tu consulta: ")
vector = embeddings.embed_query(consulta)
vector = str(vector)  
#Hacemos un select para buscar las paginas que tengan una similitud con la consulta
with conn.cursor() as cur:
    cur.execute("SELECT texto_original FROM chunks ORDER BY embedding <=> %s LIMIT 5;", (vector,))
    rows = cur.fetchall()
    print(rows)

Vamos a probarlo junto con un modelo de IA generativa

In [ ]:
#Instanciamos un modelo de chat de Azure OpenAI
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    model="gpt-4.1",
    azure_endpoint="",
    api_key="",
    api_version="2024-12-01-preview"
)

In [ ]:
#Pedimos al usuario una pregunta y la vectorizamos
consulta = input("Introduce tu consulta: ")
vector = embeddings.embed_query(consulta)
vector = str(vector)  
#Hacemos un select para buscar las paginas que tengan una similitud con la consulta
with conn.cursor() as cur:
    cur.execute("SELECT texto_original FROM chunks ORDER BY embedding <=> %s LIMIT 5;", (vector,))
    rows = cur.fetchall()
    print(rows)
#Hagamos una prueba insertando el resultado de la query en un prompt 
prompt = f"""
Eres un experto en finanzas y contabilidad. Tu tarea es analizar el siguiente texto:
{rows}

Y responder a la siguiente pregunta:
{consulta}

El usuario no es experto en finanzas y contabilidad, por lo que tu respuesta debe ser clara y concisa.
Distingue claramente entre los datos extraidos del texto y cualquier otra información que utilices para responder la pregunta.
"""

answer = llm.invoke(prompt)
print(answer.content)

### Vamos a profundizar más en lo que estamos haciendo. Añadamos observabilidad para entender como funciona cada elemento del sistema

Añadimos un par de librerias para poder visualizar de manera cómoda que está pasando.
Langsmith es el sistema de observabilidad de langchain
Logfire es el sistema de observavilidad de pydantic
Ambos tienen una versión gratuita, pero yo personalmente, prefiero logfire. Vamos a usar langsmith para generar las trazas y redirigirlas a logfire. 
Actualmente la mayoría de sistemas de observabilidad se basan en opentelemetry y son interoperables.

In [ ]:
%pip install logfire
%pip install langsmith

Configuramos logfire y las variables de OpenTelemetry

Nota: Si no recibe logs el sistema, puede ser porque los siguientes comandos conviene hacerlos antes de instanciar los llm con langchain

In [ ]:
import os
import logfire

os.environ['LANGSMITH_OTEL_ENABLED'] = 'true'
os.environ['LANGSMITH_TRACING'] = 'true'

logfire.configure(token="")

Ampliamos la consulta anterior con el comando @traceable que genera trazas sobre la entrada y salida de cualquier función. Como hemos activado las variables de langsmith, todas las llamadas a llms se trazarán automáticamente

In [ ]:
from langsmith import traceable
#Vamos a dejar la consulta fija para que los experimentos sean mas faciles de comparar
consulta = "Facturacion según mercado"
vector = embeddings.embed_query(consulta)
vector = str(vector)  
#Con la etiqueta traceable, logfire nos permite ver el codigo de la funcion y los parametros que se le pasan
@traceable
def search_chunks(vector):
    with conn.cursor() as cur:
        #Añadimos un segundo parametro al select para poder medir la similitud.
        cur.execute("SELECT texto_original,  1 - (embedding <=> %s) FROM chunks ORDER BY embedding <=> %s LIMIT 5;", (vector,vector))
        rows = cur.fetchall()
        return(rows)

def split_text(rows):
    #discard the second column and return a combined string of the first column
    return "\n".join([row[0] for row in rows])

#Hagamos una prueba insertando el resultado de la query en un prompt 
prompt = f"""
Eres un experto en finanzas y contabilidad. Tu tarea es analizar el siguiente texto:
{split_text(search_chunks(vector))}

Y responder a la siguiente pregunta:
{consulta}

El usuario no es experto en finanzas y contabilidad, por lo que tu respuesta debe ser clara y concisa.
Distingue claramente entre los datos extraidos del texto y cualquier otra información que utilices para responder la pregunta.
"""

answer = llm.invoke(prompt)
print(answer.content)

Con N=1

In [ ]:
from langsmith import traceable
#Vamos a dejar la consulta fija para que los experimentos sean mas faciles de comparar
consulta = "Facturacion según mercado"
vector = embeddings.embed_query(consulta)
vector = str(vector)  
#Con la etiqueta traceable, logfire nos permite ver el codigo de la funcion y los parametros que se le pasan
@traceable
def search_chunks(vector):
    with conn.cursor() as cur:
        #Añadimos un segundo parametro al select para poder medir la similitud.
        cur.execute("SELECT texto_original,  1 - (embedding <=> %s) FROM chunks ORDER BY embedding <=> %s LIMIT 1;", (vector,vector))
        rows = cur.fetchall()
        return(rows)

def split_text(rows):
    #discard the second column and return a combined string of the first column
    return "\n".join([row[0] for row in rows])

#Hagamos una prueba insertando el resultado de la query en un prompt 
prompt = f"""
Eres un experto en finanzas y contabilidad. Tu tarea es analizar el siguiente texto:
{split_text(search_chunks(vector))}

Y responder a la siguiente pregunta:
{consulta}

El usuario no es experto en finanzas y contabilidad, por lo que tu respuesta debe ser clara y concisa.
Distingue claramente entre los datos extraidos del texto y cualquier otra información que utilices para responder la pregunta.
"""

answer = llm.invoke(prompt)
print(answer.content)

Podemos ver que solo con una página, la respuesta es mucho menos interesante (carece de datos)
Probemos ahora con N=20

In [ ]:
from langsmith import traceable
#Vamos a dejar la consulta fija para que los experimentos sean mas faciles de comparar
consulta = "Facturacion según mercado"
vector = embeddings.embed_query(consulta)
vector = str(vector)  
#Con la etiqueta traceable, logfire nos permite ver el codigo de la funcion y los parametros que se le pasan
@traceable
def search_chunks(vector):
    with conn.cursor() as cur:
        #Añadimos un segundo parametro al select para poder medir la similitud.
        cur.execute("SELECT texto_original,  1 - (embedding <=> %s) FROM chunks ORDER BY embedding <=> %s LIMIT 20;", (vector,vector))
        rows = cur.fetchall()
        return(rows)

def split_text(rows):
    #discard the second column and return a combined string of the first column
    return "\n".join([row[0] for row in rows])

#Hagamos una prueba insertando el resultado de la query en un prompt 
prompt = f"""
Eres un experto en finanzas y contabilidad. Tu tarea es analizar el siguiente texto:
{split_text(search_chunks(vector))}

Y responder a la siguiente pregunta:
{consulta}

El usuario no es experto en finanzas y contabilidad, por lo que tu respuesta debe ser clara y concisa.
Distingue claramente entre los datos extraidos del texto y cualquier otra información que utilices para responder la pregunta.
"""

answer = llm.invoke(prompt)
print(answer.content)

Esta respuesta parece la mejor hasta ahora, pero ha aumentado un ~30% el tiempo de ejecución y el coste ha sido de aproximadamente 0.03$, respecto a 0.006$ en el caso anterior y 0.01$ en el primer experimento.


Otra consulta

In [ ]:
from langsmith import traceable
#Vamos a dejar la consulta fija para que los experimentos sean mas faciles de comparar
consulta = "Resumen de los estados financieros"
vector = embeddings.embed_query(consulta)
vector = str(vector)  
#Con la etiqueta traceable, logfire nos permite ver el codigo de la funcion y los parametros que se le pasan
@traceable
def search_chunks(vector):
    with conn.cursor() as cur:
        #Añadimos un segundo parametro al select para poder medir la similitud.
        cur.execute("SELECT texto_original,  1 - (embedding <=> %s) FROM chunks ORDER BY embedding <=> %s LIMIT 20;", (vector,vector))
        rows = cur.fetchall()
        return(rows)

def split_text(rows):
    #discard the second column and return a combined string of the first column
    return "\n".join([row[0] for row in rows])

#Hagamos una prueba insertando el resultado de la query en un prompt 
prompt = f"""
Eres un experto en finanzas y contabilidad. Tu tarea es analizar el siguiente texto:
{split_text(search_chunks(vector))}

Y responder a la siguiente pregunta:
{consulta}

El usuario no es experto en finanzas y contabilidad, por lo que tu respuesta debe ser clara y concisa.
Distingue claramente entre los datos extraidos del texto y cualquier otra información que utilices para responder la pregunta.
"""

answer = llm.invoke(prompt)
print(answer.content)

Contesta razonablemente bien. Examinando la distancia del coseno entre las distintas respuestas, podemos sacar al menos dos conclusiones:
- Entre la mejor y la peor no hay tanta diferencia, aunque el contenido es mucho mas util en los mejores resultados
- El peor resultado es mas similar a la pregunta que el mejor resultado de la pregunta anterior
Esto hace que no sea muy util implementar una estrategia de threshold (con un N mas grande, cortar todos los chunks cuya similitud < threshold ). Si lo pusieramos en 0.40, perderiamos todas las respuestas del caso anterior, si lo pusieramos en 0.20, no ganariamos nada...

¿Te has fijado que hay algunos chunks un poco raros? (Pone sindatosindato...) tratemos de entender que está pasando.


## Mejorando la calidad del sistema

Vamos a tratar de mejorar el retrieval usando un extractor mas avanzado
El pdf que hemos usado tiene sobre todo tablas y algún que otro diagrama
Vamos a intentar mejorar la conversión a texto. Algunos extractores interesantes son unestructured (propuesta de langchain) y llamaindex. Por simplificar vamos a usar la capacidad de openAI para interpretar pdfs que suele ser precisa.

In [ ]:
from openai import OpenAI
import base64
from pypdf import PdfReader, PdfWriter
import tempfile

client = OpenAI(
    api_key="",
)
def process_page(file_path: str, page_index: int) -> Pagina:
    """
    Process a single page of a PDF file and return a Pagina object.
    
    Args:
        file_path (str): Path to the file to process
        page_index (int): Index of the page to process (0-indexed)
        
    Returns:
        Pagina: A Pagina object with the processed text
    """
    
    # Read the PDF file
    reader = PdfReader(file_path)
    
    # Create a temporary PDF file with the selected page
    with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_pdf:
        writer = PdfWriter()
        writer.add_page(reader.pages[page_index])
        writer.write(temp_pdf)
        temp_pdf_path = temp_pdf.name
    
    # Encode the temporary PDF file in base64
    with open(temp_pdf_path, "rb") as f:
        base64_content = base64.b64encode(f.read()).decode('utf-8')
        base64_content = f"data:application/pdf;base64,{base64_content}"
    
    prompt = """
    Eres un experto en el análisis de documentos de contabilidad.
    Tu misión es analizar el documento de contabilidad y extraer toda la información para que pueda ser utilizada en un sistema de IA. Pon especial atención en las tablas.
    """
    
    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "input_text",
                        "text": prompt
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_file",
                        "filename": f"{file_path}_page_{page_index}.pdf",
                        "file_data": base64_content
                    }
                ]
            }
        ],
        reasoning={},
        tools=[],
        temperature=0,
        max_output_tokens=20000,
        top_p=1,
        store=True
    )
    
    # Create a Pagina object with the response text as texto_fuente
    texto_fuente = response.output[0].content[0].text
    pagina_obj = Pagina(texto_fuente=texto_fuente, pagina=page_index + 1)
    
    return pagina_obj

Con el siguiente código, podemos extraer del PDF las páginas que queramos. Cada llamada tarda ~20 segundos, luego si tenemos 300 páginas tardaremos unos 100 minutos. Si tuvieramos que hacer este proceso de manera productiva, lo haríamos asíncrono.

In [ ]:
from tqdm import tqdm
import pypdf
import pickle


ruta_pdf = "jga25-cuentas-anuales-consolidadas-2024.pdf"
with open(ruta_pdf, "rb") as file:
    reader = pypdf.PdfReader(file)
    num_pages = len(reader.pages)


for i in tqdm(range(54, 56)): 
    pagina = process_page(ruta_pdf, i)
    
    # Save each page to a pickle file
    with open(f"pagina_{i}.pkl", "wb") as pickle_file:
        pickle.dump(pagina, pickle_file)

Para ahorrarte la espera (y los tokens), te dejo el resultado en ficheros pickle en extracción_v2

In [ ]:
import pickle
import os

# Ruta al directorio que contiene los archivos pickle
ruta_directorio = "extracción_v2"

# Lista para almacenar los objetos Pagina
paginas = []

# Leemos todos los archivos pickle en el directorio
for archivo in os.listdir(ruta_directorio):
    if archivo.endswith(".pkl"):
        with open(os.path.join(ruta_directorio, archivo), "rb") as f:
            pagina = pickle.load(f)
            try:
                paginas.append(pagina[0])
            except:
                paginas.append(pagina)  
                print(f"Error al cargar el archivo {pagina}")

# Imprimimos el número de páginas leídas
print(f"Se han leído {len(paginas)} páginas")

Vectorizamos la nueva extracción

In [41]:
from tqdm.asyncio import tqdm

for pagina in tqdm(paginas, desc="Vectorizando paginas"):
    pagina.vector = embeddings.embed_query(pagina.texto_fuente)

Vectorizando paginas: 100%|██████████| 374/374 [02:47<00:00,  2.24it/s]


Creamos una nueva colección para esta extracción mejorada

In [33]:

# Create a table with a vector column
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE chunksV2 (
            id serial PRIMARY KEY,
            texto_original TEXT,
            empresa TEXT,
            numero_pagina INTEGER,
            embedding vector(3072)
        );
    """)
    conn.commit()

Insertamos los vectores

In [42]:
with conn.cursor() as cur:
    for pagina in paginas:
        cur.execute("""
            INSERT INTO chunksV2 (texto_original, empresa, numero_pagina, embedding)
            VALUES (%s, %s, %s, %s)
        """, (pagina.texto_fuente, pagina.empresa, pagina.pagina, pagina.vector))
    conn.commit()

Vamos a probarlo con una consulta de las que hemos hecho antes

In [35]:
from langsmith import traceable
#Vamos a dejar la consulta fija para que los experimentos sean mas faciles de comparar
consulta = "Hablame de los costes"
vector = embeddings.embed_query(consulta)
vector = str(vector)  
#Con la etiqueta traceable, logfire nos permite ver el codigo de la funcion y los parametros que se le pasan
@traceable
def search_chunks(vector):
    with conn.cursor() as cur:
        #Añadimos un segundo parametro al select para poder medir la similitud.
        cur.execute("SELECT texto_original,  1 - (embedding <=> %s) FROM chunksV2 ORDER BY embedding <=> %s LIMIT 20;", (vector,vector))
        rows = cur.fetchall()
        return(rows)

def split_text(rows):
    #discard the second column and return a combined string of the first column
    return "\n".join([row[0] for row in rows])

#Hagamos una prueba insertando el resultado de la query en un prompt 
prompt = f"""
Eres un experto en finanzas y contabilidad. Tu tarea es analizar el siguiente texto:
{split_text(search_chunks(vector))}

Y responder a la siguiente pregunta:
{consulta}

El usuario no es experto en finanzas y contabilidad, por lo que tu respuesta debe ser clara y concisa.
Distingue claramente entre los datos extraidos del texto y cualquier otra información que utilices para responder la pregunta.
"""

answer = llm.invoke(prompt)
print(answer.content)

Claro, aquí tienes una explicación sencilla sobre los **costes** según los datos extraídos del Informe financiero anual 2024 de Iberdrola, S.A. y sus sociedades dependientes.

---

## ¿Qué son los costes en este contexto?

En el informe, los **costes** principalmente se refieren a:
- **El valor de adquisición e inversión en los activos** de la empresa (por ejemplo, terrenos, edificios, instalaciones eléctricas, maquinaria, etc.).
- **Gastos asociados al funcionamiento general del negocio**, incluyendo personal y otros.

### 1. **Coste de los activos fijos**
- **Activos fijos**: Son los bienes que utiliza la empresa durante varios años, como plantas eléctricas, flotas, edificios, terrenos, equipamiento, etc.

**Datos extraídos (2023):**
- El total del **coste de todos los activos** (sin descontar depreciaciones) era de **137.811 millones de euros** al inicio del año y aumentó a **140.326 millones de euros** al cierre de 2023.
- Este aumento se debe principalmente a nuevas inversiones, a

Como puedes ver, la calidad de los datos tiene un impacto muy alto en la calidad del sistema final.